In [ ]:
import csv
import numpy as np
import pandas as pd

# Word2vec
from gensim.models import Word2Vec
from gensim.utils import tokenize
from unidecode import unidecode

In [ ]:
train_data = pd.read_csv("data/train.csv")

In [ ]:
# embedding dimension
n_dim = 256
pca_dim = 8

In [ ]:
sentences = [list(tokenize(s, deacc=True, lower=True)) for s in train_data['text']]
print(sentences[:5])

In [ ]:
model = Word2Vec(vector_size=n_dim, window=20, min_count=1, workers=7)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=5)

In [ ]:
# len(model.wv.vocab)

In [ ]:
embeddings = np.zeros((train_data.shape[0], n_dim))

In [ ]:
for idx in range(train_data.shape[0]):
    for s in train_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
my_pca = PCA(n_components = pca_dim)
vecs_pca = my_pca.fit_transform(embeddings)
print(vecs_pca.shape)

In [ ]:
df_new = pd.concat([train_data, pd.DataFrame(vecs_pca)], axis=1)

In [ ]:
display(df_new)

In [ ]:
df_new.to_csv("data/train_with_embeddings_pca.csv")

In [ ]:
eval_data = pd.read_csv("data/evaluation.csv")

In [ ]:
embeddings = np.zeros((eval_data.shape[0], n_dim))

In [ ]:
for idx in range(eval_data.shape[0]):
    for s in eval_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [ ]:
vecs_pca = my_pca.transform(embeddings)

In [ ]:
df_new = pd.concat([eval_data, pd.DataFrame(vecs_pca)], axis=1)

In [ ]:
display(df_new)

In [ ]:
df_new.to_csv("data/evaluation_with_embeddings_pca.csv")